In [ ]:
import os
import pyvista as pv

pv.set_jupyter_backend('static')
os.environ['DISPLAY'] = ':99.0'
os.environ['PYVISTA_OFF_SCREEM'] = 'true'

In [ ]:
export_folder = 'output'

# Load data

## 3dMD

In [ ]:
import numpy as np
from mesh4d import utils

stride = 10
fps_3dmd = round(120 / stride)
marker_3dmd = utils.load_pkl_object('output/marker_3dmd.pkl')
total_3dmd = len(marker_3dmd)

## Vicon

In [ ]:
import pandas as pd

vicon_path = '/mnt/d/knpob/4-data/20211229-DynaBreast4D/vicon/6kmh_26marker_2.csv'
df = pd.read_csv(vicon_path, skiprows=4).iloc[:, 2:].dropna(axis=1)  # M16 & M19 are dropped
df

In [ ]:
marker_vicon = df.values.reshape(len(df), -1, 3)
marker_vicon.shape

# Alignment distance

In [ ]:
from scipy.spatial import KDTree

def vicon_3dmd_dist(points_vicon, points_3dmd):
    tree = KDTree(points_3dmd)
    d, _ = tree.query(points_vicon)
    
    return d.mean()

## Before alignment

In [ ]:
from mesh4d import kps

vicon = kps.MarkerSet()
vicon.load_from_array(marker_vicon, start_time=0.0, fps=100, trans_cab=None)
vicon.interp_field()

In [ ]:
scene = pv.Plotter()
scene.open_movie(os.path.join(export_folder, 'diff_3dmd_vicon.mp4'), framerate=round(120/stride))

dist_ls = []

for frame, points_3dmd in enumerate(marker_3dmd):
    try:
        timestamp = frame / fps_3dmd
        points_vicon = vicon.get_time_coord(timestamp).get_points_coord()
        dist_ls.append(vicon_3dmd_dist(points_vicon, points_3dmd))

        scene.clear()
        scene.add_points(points_vicon, point_size=10, color='green', render_points_as_spheres=True)
        scene.add_points(points_3dmd, point_size=2, color='goldenrod', render_points_as_spheres=True)
        scene.camera_position = 'xy'
        scene.write_frame()
        
    except:
        pass

    # print progress
    percent = (frame + 1) / total_3dmd
    utils.progress_bar(percent, back_str=" exported the {}-th frame".format(frame))

scene.close()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(dist_ls, label='raw')
plt.legend()
plt.xlabel('frame id')
plt.ylabel('dist (mm)')
f"std {np.std(dist_ls)} mean {np.mean(dist_ls)} min {np.min(dist_ls)} max {np.max(dist_ls)} (mm)"

## After spatial alignment

In [ ]:
vicon = kps.MarkerSet()
trans_cab = utils.load_pkl_object('output/vicon>>3dmd.pkl')
vicon.load_from_array(marker_vicon, start_time=0.0, fps=100, trans_cab=trans_cab)
vicon.interp_field()

In [ ]:
scene = pv.Plotter()
scene.open_movie(os.path.join(export_folder, 'diff_3dmd_vicon_space.mp4'), framerate=round(120/stride))

dist_ls_space = []

for frame, points_3dmd in enumerate(marker_3dmd):
    try:
        timestamp = frame / fps_3dmd
        points_vicon = vicon.get_time_coord(timestamp).get_points_coord()
        dist_ls_space.append(vicon_3dmd_dist(points_vicon, points_3dmd))

        scene.clear()
        scene.add_points(points_vicon, point_size=10, color='green', render_points_as_spheres=True)
        scene.add_points(points_3dmd, point_size=2, color='goldenrod', render_points_as_spheres=True)
        scene.camera_position = 'xy'
        scene.write_frame()
        
    except:
        pass

    # print progress
    percent = (frame + 1) / total_3dmd
    utils.progress_bar(percent, back_str=" exported the {}-th frame".format(frame))

scene.close()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(dist_ls, label='raw')
plt.plot(dist_ls_space, label='space aligned')
plt.legend()
plt.xlabel('frame id')
plt.ylabel('dist (mm)')
f"std {np.std(dist_ls_space)} mean {np.mean(dist_ls)} min {np.min(dist_ls)} max {np.max(dist_ls)} (mm)"

# Temporal alignment